# CIFAR-10 Image Classification Report

This notebook presents a comprehensive report on building, training, and evaluating a Convolutional Neural Network (CNN) for image classification using the CIFAR-10 dataset...

**Objectives:**
- Understand the CIFAR-10 dataset and its class distribution
- Implement data preprocessing and augmentation techniques...


## 1. Adding Libraries

In [ ]:
#1. adding libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

## 2. Loading CIFAR-10 Images

In [ ]:
#2. loading cifar10 images
(train_imgs, train_lbls), (test_imgs, test_lbls) = cifar10.load_data()

## 3. Checking Image Shapes

In [ ]:
#3. testing images
print(f"Training: {train_imgs.shape}, Testing: {test_imgs.shape}")

![Training and Testing Shapes](images/1.PNG)

## 4. Defining Class Labels

In [ ]:
#4. classes for cifar10 images
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

## 5. Class Distribution Visualization

In [ ]:
#5. classcounts
counts = np.bincount(train_lbls.flatten())
plt.figure(figsize=(7,3))
sns.barplot(x=cifar_classes, y=counts)
plt.xticks(rotation=45)
plt.ylabel("NO: of images")
plt.title("Class Counts")
plt.show()

![Class Distribution](images/2.PNG)

## 6. Display Sample Images

In [ ]:
#6. display grid for samples
plt.figure(figsize=(7,7))
for idx in range(12):
    plt.subplot(3,4,idx+1)
    plt.imshow(train_imgs[idx])
    plt.title(cifar_classes[train_lbls[idx,0]])
    plt.axis('off')
plt.suptitle("Sample Images from Training Set", y=1.02)
plt.tight_layout()
plt.show()

![Sample Training Images](images/3.PNG)

## 7. Normalizing Pixel Values

In [ ]:
#7. normalizing pixel values to (0,1)
train_imgs = train_imgs.astype('float32')/255.0
test_imgs = test_imgs.astype('float32')/255.0

## 8. One-Hot Encoding Labels

In [ ]:
#8. one-hot encode labels
train_lbls = to_categorical(train_lbls, num_classes=10)
test_lbls = to_categorical(test_lbls, num_classes=10)

## 9. Setting Up Data Augmentation

In [ ]:
#9. setup of imagedatagen for training
augmenter = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='reflect',
)

## 10. Creating Data Generators

In [ ]:
#10. augmented batches gen
from sklearn.model_selection import train_test_split
train_imgs, val_imgs, train_lbls, val_lbls = train_test_split(
    train_imgs, train_lbls, test_size=0.2, random_state=42
)

train_gen = augmenter.flow(train_imgs, train_lbls, batch_size=64, shuffle=True)
val_gen = ImageDataGenerator().flow(val_imgs, val_lbls, batch_size=64)
test_gen = ImageDataGenerator().flow(test_imgs, test_lbls, batch_size=64)

## 11. Initializing CNN Model

In [ ]:
from tensorflow.keras import Model, layers

class CIFAR(Model):
    def __init__(self):
        super().__init__()
        # A layer
        self.convA = layers.Conv2D(32, 3, padding='same')
        self.bnA = layers.BatchNormalization()
        self.reluA = layers.ReLU()
        self.convA1 = layers.Conv2D(32, 3, padding='same')
        self.bnA1 = layers.BatchNormalization()
        self.reluA1 = layers.ReLU()
        self.poolA = layers.MaxPooling2D()
        self.dropA = layers.Dropout(0.1)

        # B layer
        self.convB = layers.Conv2D(64, 3, padding='same')
        self.bnB = layers.BatchNormalization()
        self.reluB = layers.ReLU()
        self.convB1 = layers.Conv2D(64, 3, padding='same')
        self.bnB1 = layers.BatchNormalization()
        self.reluB1 = layers.ReLU()
        self.poolB = layers.MaxPooling2D()
        self.dropB = layers.Dropout(0.2)

        # C layer
        self.convC = layers.Conv2D(128, 3, padding='same')
        self.bnC = layers.BatchNormalization()
        self.reluC = layers.ReLU()
        self.convC1 = layers.Conv2D(128, 3, padding='same')
        self.bnC1 = layers.BatchNormalization()
        self.reluC1 = layers.ReLU()
        self.poolC = layers.MaxPooling2D()
        self.dropC = layers.Dropout(0.3)


## 12. D Classifier Head

In [ ]:
        # D Classifier head
        self.poolD = layers.GlobalAveragePooling2D()
        self.dense2 = layers.Dense(10, activation='softmax')

## 13. Defining the Forward Pass

In [ ]:
    def call(self, x, training=False):
        x = self.convA(x)
        x = self.bnA(x, training=training)
        x = self.reluA(x)
        x = self.convA1(x)
        x = self.bnA1(x, training=training)
        x = self.reluA1(x)
        x = self.poolA(x)
        x = self.dropA(x, training=training)

        x = self.convB(x)
        x = self.bnB(x, training=training)
        x = self.reluB(x)
        x = self.convB1(x)
        x = self.bnB1(x, training=training)
        x = self.reluB1(x)
        x = self.poolB(x)
        x = self.dropB(x, training=training)

        x = self.convC(x)
        x = self.bnC(x, training=training)
        x = self.reluC(x)
        x = self.convC1(x)
        x = self.bnC1(x, training=training)
        x = self.reluC1(x)
        x = self.poolC(x)
        x = self.dropC(x, training=training)

        x = self.poolD(x)
        x = self.dense2(x)

        return x

## 14. Instantiating and Compiling the Model

In [ ]:
#14. instantiate model
model = CIFAR()

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=0.001,
    decay_steps=20000,
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

## 15. Training the Model with Callbacks

In [ ]:
#15. training the model with callbacks
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=callbacks,
)

![Training Progress](images/4.PNG)

## 16. Model Evaluation

In [ ]:
#16. evaluation
from sklearn.metrics import classification_report, confusion_matrix

pred_probs = model.predict(test_imgs, batch_size=64)
pred_labels = np.argmax(pred_probs, axis=1))
true_labels = np.argmax(test_lbls, axis=1)

print("Model Performance")
print(classification_report(true_labels, pred_labels, target_names=cifar_classes, digits=4))

![Classification Report](images/5.PNG)

## 17. Confusion Matrix Heatmap

In [ ]:
#17. confusion matrix heatmap
cm = confusion_matrix(true_labels, pred_labels)
plt.figure(figsize=(7,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='coolwarm',
            xticklabels=cifar_classes, yticklabels=cifar_classes)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix: which classes get confused!")
plt.show()

![HEATMAP](images/6.PNG)

## 18. Baseline Model Comparison

In [ ]:
#18. baseline
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

def baseline_model():
    model = Sequential()
    model.add(layers.Conv2D(64, 3, activation='relu', padding='same', input_shape=(32,32,3)))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(128, 3, activation='relu', padding='same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

base = baseline_model()
base_history = base.fit(train_gen, validation_data=val_gen, epochs=20, callbacks=callbacks)

plt.plot(history.history["val_accuracy"], label="custom CNN")
plt.plot(base_history.history["val_accuracy"], label="baseline")
plt.title("Validation Accuracy Comparison")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

![BASELINE EPOCHS](images/7.PNG)

![BASELINE VALIDATIONS](images/8.PNG)

## 19. Validation Accuracy and Loss Plots

In [ ]:
#19. results and visualizations
plt.figure(figsize=(7,7))
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Validation Loss")
plt.xlabel("Epochs")
plt.legend()
plt.show()

![Class Distribution](images/9.PNG)

## 20. Misclassified Images

In [ ]:
#20. misleading predictions
mis_index = np.where(pred_labels != np.argmax(test_lbls, axis=1))[0]
plt.figure(figsize=(7,7))
for idx in range(12):
    plt.subplot(3,4,idx+1)
    plt.imshow(test_imgs[mis_index[idx]])
    plt.title(f"True: {cifar_classes[np.argmax(test_lbls[mis_index[idx]], axis=0)]}\n"
              f"Pred: {cifar_classes[pred_labels[mis_index[idx]]]}")
    plt.axis('off')
plt.suptitle("Misclassified Images", y=1.02)
plt.tight_layout()
plt.show()

![Class Distribution](images/10.PNG)

In [ ]:
#21. Correct predictions
correct_index = np.where(pred_labels == np.argmax(test_lbls, axis=1))[0]
plt.figure(figsize=(7,7))
for idx in range(12):
    plt.subplot(3,4,idx+1)
    plt.imshow(test_imgs[correct_index[idx]])
    plt.title(f"True: {cifar_classes[np.argmax(test_lbls[correct_index[idx]], axis=0)]}\n"
              f"Pred: {cifar_classes[pred_labels[correct_index[idx]]]}")
    plt.axis('off')
plt.suptitle("Correctly Predicted Images", y=1.02)
plt.tight_layout()
plt.show()

![Class Distribution](images/11.PNG)